<a href="https://colab.research.google.com/github/myllanes/Introduction-to-Deep-Learning/blob/main/HW5_2_2_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Michael Yllanes
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import requests
import time
import torch.optim as optim
import math

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Prepare the dataset
sequence_length = 20 # Sequence

# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Encoding
class Encoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(Encoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, nhead, num_layers):
        super(CharTransformer, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = Encoding(hidden_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Embedding and positional encoding
        embedded = self.embedding(x) * math.sqrt(self.hidden_size)
        embedded = self.pos_encoder(embedded)

        # Transformer expects
        embedded = embedded.permute(1, 0, 2)

        # Transformer encoder
        transformer_out = self.transformer_encoder(embedded)

        # Get the last time step's output
        output = self.fc(transformer_out[-1, :, :])
        return output

# Hyperparameters
input_size = len(chars)
hidden_size = 120
output_size = len(chars)
learning_rate = 0.002
epochs = 5

# Model
model = CharTransformer(input_size, hidden_size, output_size, nhead=2, num_layers=1) # Number of Heads and Layers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Calculate model size
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params}")

# Training metrics storage
training_metrics = {
    'epoch_loss': [],
    'epoch_accuracy': [],
    'batch_loss': [],
    'training_time': 0
}

validation_metrics = {
    'loss': 0,
    'accuracy': 0
}

# Training loop
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    start_time = time.time()
    for epoch in range(epochs):
        epoch_loss = 0
        correct = 0
        total = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            training_metrics['batch_loss'].append(loss.item())

            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

            if batch_idx % 100 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")

        avg_loss = epoch_loss / len(train_loader)
        accuracy = correct / total
        training_metrics['epoch_loss'].append(avg_loss)
        training_metrics['epoch_accuracy'].append(accuracy)
        print(f"Epoch {epoch+1}, Average Training Loss: {avg_loss:.4f}, Training Accuracy: {accuracy:.4f}")

    training_metrics['training_time'] = time.time() - start_time
    print(f"Training time: {training_metrics['training_time']:.2f} seconds")

# Evaluation
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total_samples += target.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total_samples
    validation_metrics['loss'] = avg_loss
    validation_metrics['accuracy'] = accuracy
    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, epochs)

# Evaluate the model
evaluate(model, test_loader, criterion)

# Generate the final report
print(f"Training Duration: {training_metrics['training_time']:.2f} seconds")
print(f"Final Training Loss: {training_metrics['epoch_loss'][-1]:.4f}")
print(f"Final Training Accuracy: {training_metrics['epoch_accuracy'][-1]:.4f}")
print(f"Validation Loss: {validation_metrics['loss']:.4f}")
print(f"Validation Accuracy: {validation_metrics['accuracy']:.4f}")
print(f"Model Parameters: {total_params}")

# Computational Complexity Analysis
print(f"  - Embedding layer: ({input_size * hidden_size})")
print(f"  - Final linear layer: ({hidden_size * output_size})")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Total trainable parameters: 567913
Epoch 1, Batch 0, Loss: 4.4009
Epoch 1, Batch 100, Loss: 2.2543
Epoch 1, Batch 200, Loss: 2.6117
Epoch 1, Batch 300, Loss: 2.5063
